# Market Making (RL)

*In this notebook, we will implement a simple market making strategy using reinforcement learning. The goal of the market maker is to maximize the profit by providing liquidity to the market. The market maker will place two limit orders, one buy order and one sell order, around the mid price. The market maker will adjust the price of the limit orders based on the current inventory and the recent price movements. The market maker will also adjust the size of the limit orders based on the current inventory and the recent price movements. The market maker will use reinforcement learning to learn the optimal price and size of the limit orders.*

## Setup

In [3]:
# Data loading and preprocessing
from utils.load_data import load_data
import random
import numpy as np


seed = 123
random.seed(seed)
np.random.seed(seed)

## Data Preparation
The data used for this notebook is the [Crypto Lake website](https://crypto-lake.com/). The data contains the order book data for the "BTC/USDT" trading pair as well as the trades on the Binance exchange. It contains 20 price levels for each order book side for each timestamp(~850k) of the day "1/10/2022" plus the trades that occured on that day.


### Data Loading

In [4]:
market_event = load_data(max_depth = 10)

Missing values for book :  0 
--------------------------------------------------
Missing values for trades :  0 
--------------------------------------------------


### Data preprocessing